In [1]:
import asyncio

from pyinjective.async_client import AsyncClient
from pyinjective.client.model.pagination import PaginationOption
from pyinjective.core.network import Network
import pandas as pd 

async def sub_order(account_address) -> None:
    network = Network.local()
    client = AsyncClient(network)
    subacc_list = await client.fetch_subaccounts_list(account_address)
    return subacc_list["subaccounts"]

async def order_list(subaccount_id,market_id) -> None:
    # select network: local, testnet, mainnet
    network = Network.local()
    client = AsyncClient(network)
    skip = 0
    limit = 100
    pagination = PaginationOption(skip=skip, limit=limit)
    orders = await client.fetch_subaccount_orders_list(
        subaccount_id=subaccount_id, market_id=market_id, pagination=pagination
    )
    return orders['orders']

async def subfinder(account_address,market_id) -> None:
    all_orders =  []
    subacc_list = await sub_order(account_address)
    for i in subacc_list: 
        all_orders.extend(await order_list(i,market_id))

    df = pd.DataFrame(all_orders)
    pd.set_option('display.max_colwidth', 100)
    #print(df)
    df = df[["orderSide","subaccountId","price","quantity"]]
    df["price"] = df["price"].astype(float)/1e6
    df["quantity"] = df["quantity"].astype(float)
    df["quantity_in_$"] = df["price"] * df["quantity"]
    df = df[["orderSide","subaccountId","price","quantity_in_$"]]
    return df



/Users/rvie/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
from functions import marketId_to_ticker

ticker = "TRUMP/USDT PERP"
address = "inj1jddhqk6egwcwrwzev5wjeneht0ykmlwhcehc4z"

market_id = await marketId_to_ticker(ticker)
await subfinder(account_address=address,market_id=market_id[0])

,orderSide,subaccountId,price,quantity_in_$
0,buy,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000008,36.44,50307.242
1,buy,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000008,36.40,6861.400
2,sell,0x935b705b5943b0e1b859651d2ccf375bc96dfdd7000000000000000000000008,36.60,6899.100
